In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from pathlib import Path
import pyreadr
import os
from os.path import join, basename
import sys
from sklearn.model_selection import train_test_split
sys.path.append(os.path.abspath('../'))

from arguments import parse_arguments
from exp_components.utils import get_data_path

def get_previous_transactions_for_target(transactions, target_type, transaction_id_column='id', user_column='user', timestamp_column='timestamp', max_previous=9):
    """
    For each current transaction (i.e. with type == target_type), obtain the current transaction and up to max_previous
    transactions (of all types) from the same user that occurred before it.
    
    Parameters:
    - transactions: DataFrame containing all transaction data.
    - target_type: The target type for current transactions (e.g. "borrow").
    - transaction_id_column: Name of the transaction ID column (default 'id').
    - user_column: Name of the user ID column (default 'user').
    - timestamp_column: Name of the timestamp column (default 'timestamp').
    - max_previous: Maximum number of previous transactions to retrieve (default 9).
    
    Returns:
    - A list of tuples, each containing (current transaction Series, DataFrame of previous transactions)
      where the current transaction has type == target_type.
    """
    # Sort all transactions by user and timestamp
    transactions_sorted = transactions.sort_values(by=[user_column, timestamp_column]).reset_index(drop=True)
    
    previous_transactions = []
    # Only iterate over transactions that are of the target type (as current transaction)
    for idx, row in transactions_sorted.iterrows():
        if row.get("type") != target_type:
            continue
        current_transaction = row
        user = row[user_column]
        current_timestamp = row[timestamp_column]
        # Get transactions for the same user with timestamp less than current_timestamp (all types)
        previous = transactions_sorted[
            (transactions_sorted[user_column] == user) & 
            (transactions_sorted[timestamp_column] < current_timestamp)
        ].sort_values(by=timestamp_column, ascending=False).head(max_previous)
        previous_transactions.append((current_transaction, previous))
    
    return previous_transactions

def flatten_previous_transactions(previous_transactions, transaction_id_column='id', user_column='user', max_previous=9):
    """
    Flatten each current transaction (with its previous transactions) into a single row,
    placing the user in the first column.
    
    Parameters:
    - previous_transactions: List returned from get_previous_transactions_for_target.
    - transaction_id_column: Name of the transaction ID column (default 'id').
    - user_column: Name of the user ID column (default 'user').
    - max_previous: Maximum number of previous transactions to retrieve (default 9).
    
    Returns:
    - A flattened DataFrame with each row corresponding to a current transaction.
    """
    flattened_data = []
    for current, previous in previous_transactions:
        row = {user_column: current[user_column], transaction_id_column: current[transaction_id_column]}
        # Add features from the current transaction
        for col in current.index:
            if col not in [transaction_id_column, user_column, 'timestamp']:
                row[f"{col}_current"] = current[col]
        # Add features from the previous transactions
        for i in range(max_previous):
            if i < len(previous):
                prev_transaction = previous.iloc[i]
                for col in prev_transaction.index:
                    if col not in [transaction_id_column, user_column, 'timestamp']:
                        row[f"{col}_previous_{i+1}"] = prev_transaction[col]
            else:
                # If there are fewer than max_previous transactions, fill with NaN
                for col in current.index:
                    if col not in [transaction_id_column, user_column, 'timestamp']:
                        row[f"{col}_previous_{i+1}"] = np.nan
        flattened_data.append(row)
    
    flattened_df = pd.DataFrame(flattened_data)
    
    # Reorder columns to place the user column first
    cols = [user_column, transaction_id_column] + [col for col in flattened_df.columns if col not in [user_column, transaction_id_column]]
    flattened_df = flattened_df[cols]
    
    return flattened_df

def preprocess_flattened_data_s(flattened_data, train_data=None, max_previous=9):
    """
    Preprocess the flattened data.
    
    Parameters:
    - flattened_data: Flattened DataFrame.
    - train_data: (Optional) training data used for aligning categorical features.
    - max_previous: Maximum number of previous transactions (default 9).
    
    Returns:
    - A preprocessed DataFrame with all missing values replaced by -1.
    """
    # Drop unnecessary columns (e.g., historical 'timestamp' columns)
    features_to_drop = ['timestamp']
    transaction_columns_to_drop = [
        f"{feature}_previous_{i}" for feature in features_to_drop for i in range(1, max_previous + 1)
    ] + [f"{feature}_current" for feature in features_to_drop]
    
    flattened_data = flattened_data.drop(
        columns=[col for col in transaction_columns_to_drop if col in flattened_data.columns]
    )
    
    # Convert object type columns to category type
    for col in flattened_data.select_dtypes(include=['object']).columns:
        flattened_data[col] = flattened_data[col].astype('category')
    
    # If training data is provided, align the categorical features
    if train_data is not None:
        categorical_features = flattened_data.select_dtypes(include=['category']).columns
        for feature in categorical_features:
            flattened_data[feature] = pd.Categorical(
                flattened_data[feature], categories=train_data[feature].cat.categories
            )
    
    # Fill all missing values with -1.
    # For categorical columns, add -1 to categories before filling if necessary.
    for col in flattened_data.columns:
        if flattened_data[col].dtype.name == 'category':
            if -1 not in flattened_data[col].cat.categories:
                flattened_data[col] = flattened_data[col].cat.add_categories([-1])
            flattened_data[col] = flattened_data[col].fillna(-1)
        else:
            flattened_data[col] = flattened_data[col].fillna(-1)
    
    return flattened_data



In [ ]:
# -------------------- Main Program --------------------


target_type = "repay"

# transactions = pyreadr.read_r(f"{data_path}transactions{feature_extension}.rds")[None]
transactions = pyreadr.read_r("/data/IDEA_DeFi_Research/Data/Lending_Protocols/Aave/V2/Mainnet/transactions_user_market_time.rds")[None]
print(f"Loaded transaction data, total rows: {transactions.shape[0]})")

transactions = transactions.loc[:, ~transactions.columns.str.contains('exo')]
transactions = transactions[transactions['type'] != 'collateral']

transactions['timestamp'] = pd.to_datetime(transactions['timestamp'].astype(float), unit='s', origin='1970-01-01')

previous_transactions = get_previous_transactions_for_target(transactions, target_type=target_type)

flattened_df = flatten_previous_transactions(previous_transactions)

processed_data = preprocess_flattened_data_s(flattened_df)

train_data, test_data = train_test_split(processed_data, test_size=0.2, random_state=42)

output_dir = "data"
os.makedirs(output_dir, exist_ok=True)

train_path_rds = os.path.join(output_dir, f"train_{target_type}.rds")
test_path_rds = os.path.join(output_dir, f"test_{target_type}.rds")

pyreadr.write_rds(train_path_rds, train_data)
pyreadr.write_rds(test_path_rds, test_data)

print(f"Training data for type '{target_type}' saved to: {train_path_rds}")
print(f"Test data for type '{target_type}' saved to: {test_path_rds}")


Loaded transaction data, total rows: 1977491)


In [9]:
# 读取 CSV 文件
df = pd.read_csv('~/LTM-Encoder-Models/LTM_Structure/data/train_borrow.csv')

# 将 DataFrame 存储为 RDS 文件
pyreadr.write_rds('~/LTM-Encoder-Models/LTM_Structure/data/train_borrow.rds', df)

/tmp/ipykernel_1517812/3566275334.py:2: DtypeWarning: Columns (138,140,144,145,152,250,264,266,270,271,390,392,396,397,516,518,522,523,642,644,648,649,768,770,774,775,894,896,900,901,1020,1022,1026,1027,1146,1148,1152,1153) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('~/LTM-Encoder-Models/LTM_Structure/data/train_borrow.csv')


In [10]:
df = pd.read_csv('~/LTM-Encoder-Models/LTM_Structure/data/test_borrow.csv')

# 将 DataFrame 存储为 RDS 文件
pyreadr.write_rds('~/LTM-Encoder-Models/LTM_Structure/data/test_borrow.rds', df)

/tmp/ipykernel_1517812/2828609540.py:2: DtypeWarning: Columns (138,140,144,145,264,266,270,271,390,392,396,397,516,518,522,523,642,644,648,649,768,770,774,775,894,896,900,901,1020,1022,1026,1027,1146,1148,1152,1153) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('~/LTM-Encoder-Models/LTM_Structure/data/test_borrow.csv')
